In [6]:
import mysql.connector
import pandas as pd
import plotly.express as px
import streamlit as st


In [2]:
# Database Configuration
db_name='classicmodels'

DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "admin",
    "database": db_name,
}
# Fetch Data
conn = mysql.connector.connect(**DB_CONFIG)

query = "SHOW TABLES;"
cursor = conn.cursor()
cursor.execute(query)
tables = cursor.fetchall()
print(tables)


query = "SELECT * FROM products;"

df = pd.read_sql(query, conn)

conn.close()


[('customers',), ('employees',), ('offices',), ('orderdetails',), ('orders',), ('payments',), ('productlines',), ('products',)]


C:\Users\Shera\AppData\Local\Temp\ipykernel_31884\2809994654.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [3]:
df

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos...",5582,91.02,193.66
4,S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steer...,3252,85.68,136.00
...,...,...,...,...,...,...,...,...,...
105,S700_3505,The Titanic,Ships,1:700,Carousel DieCast Legends,"Completed model measures 19 1/2 inches long, 9...",1956,51.09,100.17
106,S700_3962,The Queen Mary,Ships,1:700,Welly Diecast Productions,Exact replica. Wood and Metal. Many extras inc...,5088,53.63,99.31
107,S700_4002,American Airlines: MD-11S,Planes,1:700,Second Gear Diecast,Polished finish. Exact replia with official lo...,8820,36.27,74.03
108,S72_1253,Boeing X-32A JSF,Planes,1:72,Motor City Art Classics,"10"" Wingspan with retractable landing gears.Co...",4857,32.77,49.66


In [4]:
def plot_graph(data, x, title, type_of_graph, y=None):
    plot_color = ["#ea5545", "#f46a9b", "#ef9b20", "#edbf33", "#ede15b", "#bdcf32", "#87bc45", "#27aeef", "#b33dc6"]


    graph_types = {
        'bar': px.bar,
        'scatter': px.scatter,
        'line': px.line,
        'box': px.box,
        'histogram': px.histogram,
        'violin': px.violin,
        'density_contour': px.density_contour,
        'density_heatmap': px.density_heatmap,
        'pie': px.pie
    }

    if type_of_graph not in graph_types:
        raise ValueError(f"Unsupported graph type. Choose from: {', '.join(graph_types.keys())}")
    

    if type_of_graph == 'violin':
        fig = graph_types[type_of_graph](data, x=x, y=y, title=title, color=x, color_discrete_sequence=plot_color)

    elif type_of_graph == 'pie':
        if y is None:
            data = data[x].value_counts().reset_index()
            data.columns = [x, 'count']
            fig = graph_types[type_of_graph](data, names=x, values='count', title=title, color_discrete_sequence=plot_color)
        else:
            fig = graph_types[type_of_graph](data, names=x, values=y, title=title, color_discrete_sequence=plot_color)
            
    elif type_of_graph == 'density_heatmap':
        if y is None:
            raise ValueError(f"'y' value must be provided for {type_of_graph} plots.")
        fig = graph_types[type_of_graph](data, x=x, y=y, title=title,color_continuous_scale='viridis')
    else:
        if y is None:
            raise ValueError(f"'y' value must be provided for {type_of_graph} plots.")
        fig = graph_types[type_of_graph](data, x=x, y=y, title=title, color_discrete_sequence=plot_color, color=x)

    fig.update_layout(
        title={'text': title, 'x': 0.05, 'xanchor': 'left'},
        plot_bgcolor='black',
        paper_bgcolor='black', 
        font=dict(color='white') 
    )
    
    if type_of_graph in ['scatter', 'line', 'barplot', 'box', 'histogram', 'violin', 'density_contour', 'density_heatmap']:
        fig.update_xaxes(showline=False, linewidth=2, linecolor='black')
        fig.update_yaxes(showline=False, linewidth=2, linecolor='black')
    else:
        fig.update_layout(showlegend=True)
    
    st.plotly_chart(fig, use_container_width=True)

    return fig


In [5]:
fig=plot_graph(df, x='productLine', y='quantityInStock', title='Sample Graph', type_of_graph='violin')
fig.show()

2025-02-22 16:30:10.431 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 16:30:10.432 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 16:30:10.433 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 16:30:10.796 
  command:

    streamlit run c:\Users\Shera\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-02-22 16:30:10.797 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
